# TP1 - Exercício 1
## Grupo 1

*   Diogo Coelho da Silva A100092
*   Pedro Miguel Ramôa Oliveira A97686

**Problema proposto:**


1. Considere a descrição da cifra A5/1 que consta no documento +Lógica Computacional: a Cifra A5/1 . Informação complementar pode ser obtida no artigo da Wikipedia. 


    Pretende-se
    1. Definir e codificar, em Z3 e usando o tipo BitVec para modelar a informação, uma FSM que descreva o gerador de chaves.
    2. Considere as seguintes eventuais propriedades de erro:
        1. ocorrência de um “burst”  $$\,\mathsf{0}^t\,$$  ($$t$$ zeros) que ocorre em   $$\,2^t\,$$ passos ou menos.
        2. ocorrência de um “burst” de tamanho $$\,t\,$$ que repete um “burst” anterior no mesmo output em $$2^{t/2}$$  passos ou menos.
        Tente codificar estas propriedades e verificar se são acessíveis a partir de um estado inicial aleatoriamente gerado.

**Proposta de resolução:**

O problema que foi apresentado tem como objetivo a criação de um horário semanal otimizado para a marcação de reuniões para uma *startup*. O objetivo é alocar eficientemente recursos como salas e tempo, considerando a disponibilidade dos colaboradores e as restrições dadas no enunciado.

Na solução proposta iremos modelar o problema de acordo com as nossas necessidades e utilizaremos um solver SCIP para encontrar uma solução otimizada.

Foram consideradas restrições dadas pelo enunciado, como por exemplo, a obrigatoriedade da presença do lider em todas as reuniões do seu projeto, a disponibilidade dos colaboradores, a partição exclusiva dos membros do projeto, a alocação de apenas um projeto por sala em cada slot e a garantia de um quorum minimo em cada reunião.









In [1]:
from z3 import *

# LFSRs como variáveis BitVec no Z3
lfsr1 = BitVec('lfsr1', 19)
lfsr2 = BitVec('lfsr2', 22)
lfsr3 = BitVec('lfsr3', 23)

# Taps de feedback para cada LFSR
def step_lfsr1(lfsr):
    feedback = (lfsr >> 18) ^ (lfsr >> 17) ^ (lfsr >> 16) ^ (lfsr >> 13)
    return LShR(lfsr, 1) | (feedback << 18)

def step_lfsr2(lfsr):
    feedback = (lfsr >> 21) ^ (lfsr >> 20)
    return LShR(lfsr, 1) | (feedback << 21)

def step_lfsr3(lfsr):
    feedback = (lfsr >> 22) ^ (lfsr >> 21) ^ (lfsr >> 20) ^ (lfsr >> 7)
    return LShR(lfsr, 1) | (feedback << 22)


In [2]:
def detect_burst_of_zeros(solver, output_stream, t):
    zero_burst = [output_stream[i] == 0 for i in range(t)]
    solver.add(Or(zero_burst))  # Adiciona a restrição que verifica se há um burst de zeros


In [3]:
def detect_repeated_burst(solver, output_stream, t):
    # Verifica se o burst se repete no intervalo de 2^(t/2) passos
    for i in range(2**(t//2)):
        for j in range(i + 1, 2**(t//2)):
            if solver.check(output_stream[i:i+t] == output_stream[j:j+t]) == sat:
                return True
    return False


In [4]:
def run_a5_1_simulation(t):
    solver = Solver()

    # Definir o estado inicial aleatório dos LFSRs
    lfsr1_initial = BitVec('lfsr1_initial', 19)
    lfsr2_initial = BitVec('lfsr2_initial', 22)
    lfsr3_initial = BitVec('lfsr3_initial', 23)
    
    solver.add(lfsr1_initial >= 0, lfsr1_initial < 2**19)
    solver.add(lfsr2_initial >= 0, lfsr2_initial < 2**22)
    solver.add(lfsr3_initial >= 0, lfsr3_initial < 2**23)
    
    # Aplicando a regra da maioria e avançando os LFSRs
    output_stream = []
    for step in range(2**t):
        majority = majority_bit(lfsr1_initial, lfsr2_initial, lfsr3_initial)
        output_stream.append(majority)
        lfsr1_initial = If(majority == 1, step_lfsr1(lfsr1_initial), lfsr1_initial)
        lfsr2_initial = If(majority == 1, step_lfsr2(lfsr2_initial), lfsr2_initial)
        lfsr3_initial = If(majority == 1, step_lfsr3(lfsr3_initial), lfsr3_initial)
    
    # Verificar a propriedade de burst de zeros
    detect_burst_of_zeros(solver, output_stream, t)

    # Verificar a propriedade de repetição de burst
    detect_repeated_burst(solver, output_stream, t)

    # Tentar satisfazer as restrições e verificar se a propriedade ocorre
    if solver.check() == sat:
        print("Propriedade atingível!")
    else:
        print("Propriedade não atingível.")
